In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys

import pandas as pd
import numpy as np

import time

import torch.nn.functional as F
import torch

import copy

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
os.makedirs('results/unrolling-sgd', exist_ok=True)

# finetune

In [16]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # prepare model
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L94
    M = copy.deepcopy(model)
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L150
    M_unlearned = copy.deepcopy(model)
        
    M_unlearned.train()
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L131
    error = torch.nn.CrossEntropyLoss()
    
    forget_loader = torch.utils.data.DataLoader(forget_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):    
        
        # train
        
        train_time = 0
        
        start_time = time.time()
        
        for x, y in forget_loader:

            output = M_unlearned(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            
            # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L163
            loss.backward(retain_graph=True)
            grads = torch.autograd.grad(loss, [param for param in M_unlearned.parameters()], create_graph = True)
            
            old_params = {}
            for i, (name, params) in enumerate(M.named_parameters()):
                    old_params[name] = params.clone()
                    old_params[name] += LR * grads[i]
            for name, params in M_unlearned.named_parameters():
                    params.data.copy_(old_params[name])
            
            train_time += time.time() - start_time
            
            start_time = time.time()
            
        train_times.append(train_time)
        
        # test
            
        M_unlearned.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(M_unlearned.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [17]:
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L49
LR = 0.1

In [18]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    model.load_state_dict(torch.load('./weights/original/005.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/unrolling-sgd/finetune/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [19]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/unrolling-sgd/finetune.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1        0.114819   0.098697  0.097943    0.097892
           2        0.115559   0.098697  0.097943    0.097892
           3        0.114927   0.098697  0.097943    0.097892
           4        0.113807   0.098697  0.097943    0.097892
           5        0.115245   0.098697  0.097943    0.097892
10         1        1.623883   0.098711  0.097943    0.098116
           2        1.676104   0.098711  0.097943    0.098116
           3        1.707427   0.098711  0.097943    0.098116
           4        1.522140   0.098711  0.097943    0.098116
           5        1.667175   0.098711  0.097943    0.098116
20         1        3.686478   0.098271  0.097943    0.099337
           2        3.319183   0.098271  0.097943    0.099337
           3        3.514916   0.098271  0.097943    0.099337
           4        3.503710   0.098271  0.097943    0.099337
           5        3.320124   0.098271  0.097943    0.099337
30         1        5.456588   0.098558  0.097943    0.098637
           2        5.144800   0.098558  0.097943    0.098637
           3        5.249419   0.098558  0.097943    0.098637
           4        5.227467   0.098558  0.097943    0.098637
           5        5.495168   0.098558  0.097943    0.098637
40         1        7.273762   0.098389  0.097943    0.098836
           2        7.052521   0.098389  0.097943    0.098836
           3        6.937870   0.098389  0.097943    0.098836
           4        7.113247   0.098389  0.097943    0.098836
           5        6.694447   0.098389  0.097943    0.098836
50         1        9.331224   0.098714  0.097943    0.098521
           2        8.598808   0.098714  0.097943    0.098521
           3        8.630263   0.098714  0.097943    0.098521
           4        8.774745   0.098714  0.097943    0.098521
           5        8.850331   0.098714  0.097943    0.098521
60         1       11.369717   0.099542  0.097943    0.098118
           2       10.553390   0.099542  0.097943    0.098118
           3       10.570448   0.099542  0.097943    0.098118
           4       10.481780   0.099542  0.097943    0.098118
           5       10.458821   0.099542  0.097943    0.098118
70         1       13.416203   0.098857  0.097943    0.098536
           2       12.215645   0.098857  0.097943    0.098536
           3       12.384325   0.098857  0.097943    0.098536
           4       12.393462   0.098857  0.097943    0.098536
           5       12.267377   0.098857  0.097943    0.098536
80         1       15.650421   0.099500  0.097943    0.098421
           2       14.079730   0.099500  0.097943    0.098421
           3       14.104240   0.099500  0.097943    0.098421
           4       14.034564   0.099500  0.097943    0.098421
           5       14.322129   0.099500  0.097943    0.098421
90         1       17.752018   0.097739  0.097943    0.098669
           2       15.812947   0.097739  0.097943    0.098669
           3       16.177165   0.097739  0.097943    0.098669
           4       16.289349   0.097739  0.097943    0.098669
           5       15.677236   0.097739  0.097943    0.098669
99         1       18.545673   0.095943  0.097943    0.098589
           2       17.629697   0.095943  0.097943    0.098589
           3       17.272803   0.095943  0.097943    0.098589
           4       17.738442   0.095943  0.097943    0.098589
           5       17.719019   0.095943  0.097943    0.098589

# correlation

In [5]:
def std_loss(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()
    avg_std = torch.sum(torch.std(x, dim=1))/(len(x.view(-1)))
    loss = loss + STD_REG*avg_std
    return loss

def my_cross_entropy(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()

    #x is (N,C)
    N,C = x.shape
    p = F.softmax(x,1)
    hessian_loss = torch.sum(p *(1-p),dim =1)
    hessian_loss = hessian_loss.mean()

    loss = loss + STD_REG * hessian_loss
    return loss

# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L130

loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L161

    M = copy.deepcopy(model)

    M.train()
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L172
    optimizer = torch.optim.SGD(M.parameters(), lr=LR, weight_decay = L2)
    
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    forget_loader = torch.utils.data.DataLoader(forget_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        train_time = 0
        
        start_time = time.time()
        
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L200
        
        # only update M:
        
        M.train()
        
        for x, y in forget_loader:
        
            optimizer.zero_grad()
            
            outputs_M = M(x.cuda())
            
            targets = y.cuda()
        
            if LOSS_FUNC == 'hess':
                loss_M = my_cross_entropy(outputs_M, targets)
            if LOSS_FUNC =='std':
                loss_M = std_loss(outputs_M, targets)
            if LOSS_FUNC =='regular':
                loss_M = loss_fn(outputs_M, targets)
                
            loss_M.backward()
            optimizer.step()
        
        # now we also want to compute the gradient:
        
        grad_list = []
        
        for x, y in train_loader:

            output_grad = M(x.cuda())
            
            label = y.cuda()
            
            if LOSS_FUNC == 'hess':
                loss_grad = my_cross_entropy(output_grad, label)
            if LOSS_FUNC =='std':
                loss_grad = std_loss(output_grad, label)
            if LOSS_FUNC =='regular':
                loss_grad = loss_fn(output_grad, label)
            
            loss_grad.backward(retain_graph=True)
            grads = torch.autograd.grad(loss_grad, [param for param in M.parameters()], create_graph = True)
            # !!! modified !!! original code cause OOM
            # grad_list.append(grads)
            grad_list.append([i.detach().cpu() for i in grads])
        
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L227
            
        # update both M and M'
        
        for x, y in forget_loader:
        
            optimizer.zero_grad()
            
            outputs_M = M(x.cuda())
            
            targets = y.cuda()
        
            if LOSS_FUNC == 'hess':
                loss_M = my_cross_entropy(outputs_M, targets)
            if LOSS_FUNC =='std':
                loss_M = std_loss(outputs_M, targets)
            if LOSS_FUNC =='regular':
                loss_M = loss_fn(outputs_M, targets)
                
            loss_M.backward()
            optimizer.step()
            
        # Now, get M''_(N+t)
            
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L270
        
        M_unlearned = copy.deepcopy(M)
        
        old_params = {}
        for i, (name, params) in enumerate(M.named_parameters()):
            old_params[name] = params.clone()
            for grads in grad_list:
                old_params[name] += LR * grads[i].cuda()
        for name, params in M_unlearned.named_parameters():
            params.data.copy_(old_params[name])
        
            
        train_time += time.time() - start_time

        start_time = time.time()
            
        train_times.append(train_time)
        
        # test
            
        M_unlearned.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(M_unlearned.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [13]:
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L167
LR = 0.01
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L72
L2 = 0.
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L71
STD_REG = 0.

LOSS_FUNC = 'hess'

In [14]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    model.load_state_dict(torch.load('./weights/original/005.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/unrolling-sgd/correlation/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [15]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/unrolling-sgd/correlation.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       24.607036   0.112177  0.113518    0.114521
           2       22.409146   0.104385  0.102736    0.102755
           3       21.495975   0.090805  0.089357    0.089075
           4       21.650668   0.102231  0.101038    0.100841
           5       21.699512   0.098176  0.094549    0.094346
10         1       22.453834   0.112430  0.113518    0.112837
           2       21.562140   0.096620  0.098243    0.100299
           3       22.439224   0.102599  0.103734    0.105352
           4       21.875947   0.094676  0.096745    0.098802
           5       22.301778   0.096564  0.098243    0.100611
20         1       22.122407   0.064917  0.068091    0.063227
           2       22.292046   0.102146  0.101038    0.101744
           3       22.070628   0.132979  0.135483    0.138399
           4       22.459756   0.126500  0.121106    0.125454
           5       22.792111   0.142312  0.140176    0.136537
30         1       22.798678   0.101128  0.101038    0.103382
           2       22.763214   0.103913  0.102736    0.104488
           3       22.449966   0.113600  0.116713    0.117330
           4       22.867168   0.106245  0.105431    0.104274
           5       23.421859   0.099058  0.093850    0.096675
40         1       23.278412   0.188500  0.191394    0.191792
           2       22.885220   0.091028  0.089257    0.089311
           3       23.720358   0.123417  0.126198    0.126176
           4       23.848705   0.104889  0.102736    0.103393
           5       22.565082   0.097083  0.098243    0.097925
50         1       23.553720   0.097148  0.098243    0.097697
           2       24.334825   0.098647  0.101438    0.100969
           3       23.938523   0.178738  0.187200    0.183453
           4       24.299111   0.101912  0.101038    0.102068
           5       24.088158   0.098981  0.102037    0.097922
60         1       24.448012   0.101708  0.101038    0.102182
           2       23.686167   0.229000  0.225739    0.225726
           3       23.894461   0.191500  0.193191    0.193976
           4       23.358061   0.115500  0.111322    0.113547
           5       23.952212   0.149083  0.144069    0.151752
70         1       24.385867   0.198657  0.202676    0.205028
           2       24.623375   0.101965  0.098043    0.097632
           3       23.974000   0.143928  0.144868    0.148524
           4       24.114030   0.272647  0.264577    0.260859
           5       24.420247   0.551565  0.561901    0.552045
80         1       24.597216   0.172333  0.184006    0.183053
           2       24.288840   0.350333  0.366414    0.358867
           3       24.280944   0.115917  0.126398    0.126534
           4       24.933414   0.249250  0.232129    0.238641
           5       24.853089   0.103500  0.101038    0.101713
90         1       25.026492   0.534242  0.517971    0.518897
           2       25.146636   0.595911  0.604832    0.600996
           3       25.001879   0.287899  0.269369    0.270334
           4       24.685071   0.660406  0.674022    0.670399
           5       25.280702   0.511469  0.502496    0.496939
99         1       25.164451   0.095943  0.097943    0.098589
           2       25.212517   0.095943  0.097943    0.098589
           3       25.784373   0.095943  0.097943    0.098589
           4       25.652189   0.095943  0.097943    0.098589
           5       25.334640   0.095943  0.097943    0.098589